In [212]:
import emoji 
import json
import datetime
import re
from collections import Counter
import timeit
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv

# data by https://www.kaggle.com/soaxelbrooke/twitter-basic-seq2seq/data
file_path = "C:\\Users\\Wilm\\Documents\\GitHub\\MSc\\twcs\\twcs.csv"

In [213]:
df = pd.read_csv("C:\\Users\\Wilm\\Documents\\GitHub\\MSc\\twcs\\twcs.csv")

In [3]:
df.head(3)

tweet_id   author_id  inbound                      created_at  \
0         1  sprintcare    False  Tue Oct 31 22:10:47 +0000 2017   
1         2      115712     True  Tue Oct 31 22:11:45 +0000 2017   
2         3      115712     True  Tue Oct 31 22:08:27 +0000 2017   

                                                text response_tweet_id  \
0  @115712 I understand. I would like to assist y...                 2   
1      @sprintcare and how do you propose we do that               NaN   
2  @sprintcare I have sent several private messag...                 1   

   in_response_to_tweet_id  
0                      3.0  
1                      1.0  
2                      4.0

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
tweet_id                   int64
author_id                  object
inbound                    bool
created_at                 object
text                       object
response_tweet_id          object
in_response_to_tweet_id    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [214]:
emojis = []
for i in range(0,len(df)):
    emojis.append(''.join(c for c in df["text"].loc[i] if c in emoji.UNICODE_EMOJI))

In [215]:
Counter(emojis).most_common()

[('', 2617149),
 ('😊', 7356),
 ('👍', 7320),
 ('😡', 6164),
 ('🤔', 5353),
 ('😂', 4786),
 ('🙂', 4572),
 ('🙄', 4451),
 ('😭', 3579),
 ('😉', 3532),
 ('❤', 3104),
 ('😩', 2925),
 ('😒', 2731),
 ('😞', 2364),
 ('😁', 2155),
 ('😔', 2122),
 ('😀', 2085),
 ('☹', 2084),
 ('😍', 1985),
 ('😃', 1873),
 ('😢', 1858),
 ('🙃', 1794),
 ('😕', 1694),
 ('🙏', 1604),
 ('✈', 1573),
 ('👌', 1511),
 ('😤', 1461),
 ('😑', 1442),
 ('☺', 1400),
 ('👎', 1370),
 ('🙌', 1228),
 ('😠', 1193),
 ('💚', 1176),
 ('😐', 1158),
 ('😄', 1132),
 ('😂😂', 1127),
 ('🤦♀', 1125),
 ('😡😡😡', 1076),
 ('🤷♀', 980),
 ('🤦♂', 968),
 ('😡😡', 932),
 ('🙁', 901),
 ('😂😂😂', 885),
 ('😫', 857),
 ('😬', 846),
 ('😅', 759),
 ('😎', 758),
 ('😱', 747),
 ('👀', 734),
 ('😳', 719),
 ('👋', 701),
 ('\U0001f92c', 697),
 ('😋', 696),
 ('😘', 664),
 ('👏', 654),
 ('😭😭😭', 652),
 ('✅', 646),
 ('🙈', 639),
 ('🤗', 617),
 ('😭😭', 598),
 ('🤷♂', 594),
 ('😣', 564),
 ('🥕', 561),
 ('😟', 548),
 ('😜', 536),
 ('😧', 534),
 ('🖕', 529),
 ('✌', 503),
 ('🤢', 501),
 ('😏', 497),
 ('🤞', 492),
 ('🤣', 476),
 (

In [245]:
top_20_emoj_df = pd.DataFrame(Counter(emojis).most_common(20), columns=['emoji', 'Anzahl'])
#top_20_emoj_df.cumsum()
top_20_emoj_wo_zero_df = top_20_emoj_df[1:]
top_20_emoj_wo_zero_df

emoji  Anzahl
1      😊    7356
2      👍    7320
3      😡    6164
4      🤔    5353
5      😂    4786
6      🙂    4572
7      🙄    4451
8      😭    3579
9      😉    3532
10     ❤    3104
11     😩    2925
12     😒    2731
13     😞    2364
14     😁    2155
15     😔    2122
16     😀    2085
17     ☹    2084
18     😍    1985
19     😃    1873

In [217]:
#df_filtered = df[df['text'].str.contains([e for e in single_emoj_list])] # beinhaltet nicht '😊'
#df[~df['text'].isin(single_emoj_list)]
tweets_w_emoj_df = df[df['text'].str.contains('|'.join(single_emoj_list))] # eines der emoji aus single_emoj_list -> match auch mit gruppe aus emoji
tweets_w_emoj_df

31         somebody from @VerizonSupport please help meee...
62         @ChipotleTweets messed up today and didn’t giv...
77         @ChipotleTweets Thank you @ChipotleTweets for ...
78         So frustrated with @ChipotleTweets 😡 Ordered d...
91                                    @115741 Noted 😊 -Becky
95         Considering walking to @ChipotleTweets in my l...
119        Excellent Halloweening @marksandspencer 👌🏼 #co...
187                                 amazonのfireTVstickが見れない😢
246        @AirAsiaSupport Hi! i would like to know on my...
263        @115801 @115802 @AirAsiaSupport You'll be even...
272        @Morrisons Aahhrrgh! Keep mum, I’m still watch...
274                @Morrisons Aww, Sophie deserved winner 👏😄
275        Fresh cream palmier @Morrisons bakery Bellshil...
311        whats wrong w/ amazon leavin shit outside like...
340        @115821 @AmazonHelp why is my order at my loca...
366        @115840 ¡Que lindoooooooooo! 😍🐶🎃 ^JJ https://t...
408        @AppleSupport

In [13]:
# ist "start", "initialer" tweet für dialog

"""
tweet_id: A unique, anonymized ID for the Tweet. Referenced by response_tweet_id and in_response_to_tweet_id
response_tweet_id: IDs of tweets that are responses to this tweet, comma-separated.
in_response_to_tweet_id: ID of the tweet this tweet is in response to, if any.
"""
df[df['tweet_id']==2]

tweet_id author_id  inbound                      created_at  \
1         2    115712     True  Tue Oct 31 22:11:45 +0000 2017   

                                            text response_tweet_id  \
1  @sprintcare and how do you propose we do that               NaN   

   in_response_to_tweet_id  
1                      1.0

In [8]:
# tweets die auf einen anderen tweet antworten 
tweets_w_emoj_df[tweets_w_emoj_df['in_response_to_tweet_id']==171.0]

tweet_id       author_id  inbound                      created_at  \
91       170  ChipotleTweets    False  Tue Oct 31 22:01:00 +0000 2017   

                      text response_tweet_id  in_response_to_tweet_id  
91  @115741 Noted 😊 -Becky               NaN                    171.0

In [10]:
# tweets_w_emoj_df.info() = 135157 entries

# nur tweets mit emoji aus df nehmen
tweets_w_in_resp_to_tweetID_df = tweets_w_emoj_df[~tweets_w_emoj_df['in_response_to_tweet_id'].isnull()]

response_tweets_df = tweets_w_in_resp_to_tweetID_df[~tweets_w_in_resp_to_tweetID_df['response_tweet_id'].isnull()] 
initial_tweets_df = tweets_w_emoj_df[tweets_w_emoj_df['in_response_to_tweet_id'].isnull()]

In [105]:
# finding all tweets that response to initial tweet

#dialogues_1vs1 = [conv1[initial_tweet, response_tweet]]
#dialogues_1vs = {conv1: {initial_tweet: str(), response_tweet: str()}}
#dialogues_1vsN = conv2: initial_tweet, reponse_tweet1, reponse_tweetN
    
dialogues_1vs1 = []
    
for index, row in initial_tweets_df.iterrows():
    response_tweet_list = [x.strip() for x in row['response_tweet_id'].split(',')]
    for response_tweet_id in response_tweet_list:
        response_tweet_id = int(response_tweet_id)
        if len(response_tweet_list) == 1: # conversation besteht lediglich aus zwei aussagen (initaler und antwortender tweet)
        #print('response_tweet_id: ', response_tweet_id, type(response_tweet_id))
            for tweet_id in response_tweets_df['tweet_id']:
                #print('tweet_id: ', tweet_id, type(tweet_id))
                if response_tweet_id == tweet_id: 
                    #print('\nConversation\n')
                    #print('initial_tweet')
                    #print(row['tweet_id'], row['text'])
                    #print(row['text'])
                    initial_tweet = row['text']

                    #print('response_tweet')
                    response_tweet = response_tweets_df.loc[response_tweets_df['tweet_id']== response_tweet_id]['text']
                    #print(response_tweet.values[0])
                    response_tweet = response_tweets_df.loc[response_tweets_df['tweet_id']== response_tweet_id]['text']
                    dialogues_1vs1.append([initial_tweet, response_tweet.values[0]])
print('done')
        #if len(response_tweet_list) > 1: # wenn conversation aus mehr als einem response_tweet besteht
        

done


In [150]:
# trash --> nicht sicher
#response_tweets_df.head(2)
#initial_tweets_df.head(10)

#len(dialogues_1vs1) # = 1346 --> mehrere Sprachen

dialogues_1vs1_lang_en = [] 
from langid.langid import LanguageIdentifier, model
identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
for conv in dialogues_1vs1:
    conversation = conv[0] + ' ' + conv[1]
    langid = identifier.classify(conversation)[0] # get the langid e.g.: "en" or "de"
    langid_likelihood = identifier.classify(conversation)[1] # e.g. "0.232234" or best "1.0"
    if langid == 'en':
        dialogues_1vs1_lang_en.append([conv[0],conv[1]]) # generates list about all "en" conversations
    

In [153]:
# conversationen nur auf englisch
# len(dialogues_1vs1_lang_en) # == 1032
dialogues_1vs1_lang_en

[['My flight back to SF just got upgraded 🙌🏾 thanks @Delta! 😊',
  '@117180 Congrats on the upgrade, Davina! Enjoy! ✈💺✈ *ACJ'],
 ['@115858 what in the world is this: A ? When I️ type I...you messing with my money 🙄😒 can’t send any emails from my phone!',
  '@115858 @115858 you see?!!!!! I️ &lt;—- that’s a bug...please fix 😒'],
 ['sooooooo who gots some spooky deals because your girl is trying to eat good with $14 🙃',
  'Sho’ did get me my $3 bowl as a chola 😂 @ChipotleTweets thanks aly!! https://t.co/GrgdHt9zC9'],
 ['Practising my Spanish 🇪🇸 alphabet pronunciation on the 06:48 @VirginTrains  keep getting stuck on G 🙄😁#nottalkingtomyself',
  '@120647 You can do it, I have faith in you 😀 . ^PA'],
 ['Wondering if @O2 will give me a cheeky free upgrade with 3 months to go on my contract for being such a loyal customer 🙏🏻😉',
  "@120821 Hi Joanne 👋 😉 Do you have something in mind you'd like? Have you already spoken to our upgrade team over the phone?"],
 ['@O2 \nTrying to redeem Garmin offer 

In [218]:
emojis_in_dialogues_1vs1_lang_en = []
for initial_tweet, response_tweet in dialogues_1vs1_lang_en:
    emojis_in_dialogues_1vs1_lang_en.append(''.join(c for c in initial_tweet if c in emoji.UNICODE_EMOJI))
    emojis_in_dialogues_1vs1_lang_en.append(''.join(c for c in response_tweet if c in emoji.UNICODE_EMOJI)) 

In [219]:
Counter(emojis_in_dialogues_1vs1_lang_en).most_common(40) # vorkommen auch von dopplungen

[('🤔', 92),
 ('😂', 75),
 ('👍', 75),
 ('😊', 73),
 ('😉', 70),
 ('😭', 69),
 ('😡', 66),
 ('😩', 61),
 ('🙃', 51),
 ('😞', 49),
 ('🙄', 48),
 ('😀', 46),
 ('❤', 45),
 ('🙂', 41),
 ('😒', 39),
 ('😁', 39),
 ('😢', 37),
 ('😕', 34),
 ('😔', 32),
 ('☹', 29),
 ('😍', 29),
 ('👌', 26),
 ('😂😂', 25),
 ('🙌', 24),
 ('😐', 22),
 ('😤', 21),
 ('🙏', 21),
 ('😡😡', 20),
 ('😃', 19),
 ('💚', 19),
 ('😑', 18),
 ('😄', 17),
 ('😠', 15),
 ('😂😂😂', 14),
 ('😡😡😡', 13),
 ('👎', 12),
 ('✈', 12),
 ('😭😭😭', 11),
 ('😭😭', 9),
 ('🙄🙄', 8)]

In [175]:
len(emojis_in_dialogues_1vs1_lang_en)

2064

In [220]:
# emoji ohne dopplungen
emoji_counter = Counter(emojis_in_dialogues_1vs1_lang_en).most_common()
emojis_in_dialogues_1vs1_lang_en_2 = []

for item in emojis_in_dialogues_1vs1_lang_en:
    if len(item) > 1 and item == item[::-1]:
        emojis_in_dialogues_1vs1_lang_en_2.append(list(set(item))[0])
    else:
        emojis_in_dialogues_1vs1_lang_en_2.append(item)
emojis_in_dialogues_1vs1_lang_en_2

['🙌😊',
 '💺',
 '🙄😒',
 '😒',
 '🙃',
 '😂',
 '🙄😁',
 '😀',
 '🙏😉',
 '👋😉',
 '😢',
 '☹',
 '😩',
 '😤',
 '🙄',
 '😞',
 '😭😭😭😭💔',
 '😞',
 '🙌🍵',
 '😁',
 '🙄',
 '😍',
 '😞',
 '☹',
 '😁❤',
 '😁',
 '🤔😠💩',
 '😔⬇',
 '😂😁',
 '😂😂😂😂😘💜',
 '🙏',
 '🍕🍕🍕😍😍😍',
 '😢',
 '🤔💭',
 '🙈❤',
 '😍',
 '❤',
 '😞✉',
 '😁',
 '😊',
 '🤔',
 '😢',
 '🙃',
 '🤔',
 '😔',
 '😢',
 '😒',
 '😢',
 '😠',
 '😞',
 '😞',
 '😞',
 '😂',
 '😂',
 '🤔',
 '😕',
 '😭',
 '😔',
 '😍',
 '👍',
 '🤔',
 '🤔',
 '😒',
 '😉',
 '😊',
 '👍📩',
 '😤',
 '🤔',
 '😊',
 '😉😂',
 '🤔',
 '😏👍🎅🎄',
 '😑',
 '😐',
 '👍',
 '😉',
 '🙊😂',
 '🙉😁',
 '😔',
 '❤',
 '👍',
 '😀☑❤',
 '™👌👌👌',
 '🙌',
 '👌',
 '🙌🎄',
 '😡',
 '😞',
 '😭',
 '😢',
 '😡',
 '😡',
 '😒',
 '😩',
 '😩',
 '😭',
 '😂',
 '😉',
 '🙃',
 '😂',
 '🤔🤔🤔😂😶',
 '😀',
 '😡',
 '😡',
 '😊',
 '🙂',
 '😒',
 '🙂',
 '😉',
 '😉',
 '😤',
 '😒',
 '😁',
 '😍',
 '👍',
 '😊',
 '😡',
 '😑',
 '👍',
 '😑',
 '😂',
 '😉🤘',
 '😕',
 '🤔',
 '😠',
 '😡',
 '😝✈',
 '😊✈',
 '😃',
 '😊',
 '👎',
 '🤦♀😔',
 '🙃',
 '😢',
 '😎😃💖😃',
 '😃',
 '😩',
 '😤',
 '😩',
 '👀😍',
 '🙏',
 '🙏',
 '🤔',
 '🙂',
 '😃',
 '😎✈',
 '😕',
 '🙂',
 '🙃',
 '😔',
 '🤔',
 '🤔😩',
 '😂',
 '💚',
 '🤞❤',
 '😁',
 '🖕😡',


In [221]:
# die TOP emoji 
# wobei Palindrome als ein emoji zu dem reduzierten emoji zählen

Counter(emojis_in_dialogues_1vs1_lang_en_2).most_common()

[('😂', 122),
 ('😡', 106),
 ('🤔', 103),
 ('😭', 94),
 ('👍', 79),
 ('😊', 78),
 ('😉', 75),
 ('😩', 70),
 ('🙃', 65),
 ('🙄', 65),
 ('😀', 49),
 ('😞', 49),
 ('❤', 49),
 ('😁', 44),
 ('😍', 44),
 ('🙂', 44),
 ('😒', 42),
 ('😢', 38),
 ('😕', 35),
 ('😤', 33),
 ('😔', 33),
 ('☹', 31),
 ('👌', 28),
 ('🙏', 27),
 ('🙌', 26),
 ('😐', 24),
 ('😑', 21),
 ('😠', 20),
 ('😃', 20),
 ('👎', 20),
 ('💚', 20),
 ('😄', 18),
 ('✈', 13),
 ('☺', 8),
 ('😞✉', 5),
 ('😔⬇', 3),
 ('👍😊', 3),
 ('😁👍', 3),
 ('😉✈', 3),
 ('🙌😊', 2),
 ('🤔😩', 2),
 ('😭😩', 2),
 ('😱😢', 2),
 ('😀👍', 2),
 ('😍😊', 2),
 ('😂😂❤', 2),
 ('👍✈', 2),
 ('❤🙏', 2),
 ('🙌☕', 2),
 ('😔🤞', 2),
 ('😂🙈', 2),
 ('😠😡', 2),
 ('😂🙌', 2),
 ('😒😡', 2),
 ('😤😩', 2),
 ('🙈😂', 2),
 ('😡😤', 2),
 ('😩😭', 2),
 ('😭💔', 2),
 ('🙄😭', 2),
 ('😉👍', 2),
 ('💺', 1),
 ('🙄😒', 1),
 ('🙄😁', 1),
 ('🙏😉', 1),
 ('👋😉', 1),
 ('😭😭😭😭💔', 1),
 ('🙌🍵', 1),
 ('😁❤', 1),
 ('🤔😠💩', 1),
 ('😂😁', 1),
 ('😂😂😂😂😘💜', 1),
 ('🍕🍕🍕😍😍😍', 1),
 ('🤔💭', 1),
 ('🙈❤', 1),
 ('👍📩', 1),
 ('😉😂', 1),
 ('😏👍🎅🎄', 1),
 ('🙊😂', 1),
 ('🙉😁', 1),
 ('😀☑❤', 1),
 ('™👌👌👌', 1

In [11]:
# tweets mit emoji und kein NaN in in_response_to_tweet_id
tweets_w_emoj_df = df.loc[~df['in_response_to_tweet_id'].isnull()] 
# konvertiert von float zu int
response_df = df['in_response_to_tweet_id'].astype('int')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
### Spielwiese

# how to: plot combined figures (emoji occurence in conversations)

# libraries
import numpy as np
import matplotlib.pyplot as plt
 
# set width of bar
barWidth = 0.25
 
# set height of bar
bars1 = [12, 30, 1, 8, 22]
bars2 = [28, 6, 16, 5, 10]
bars3 = [29, 3, 24, 25, 17]
 
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
 
# Make the plot
plt.bar(r1, bars1, color='#7f6d5f', width=barWidth, edgecolor='white', label='var1')
plt.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='var2')
plt.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='var3')
 
# Add xticks on the middle of the group bars
plt.xlabel('TOP 5 Emoji', fontweight='bold')
plt.ylabel('Anzahl', fontweight = 'bold')
plt.xticks([r + barWidth for r in range(len(bars1))], ['😂', 'B', 'C', 'D', 'E'])
 
# Create legend & Show graphic
plt.legend()
plt.show()

In [ ]:
# Stichworte vom Treffen am 29.05.2018
# 1. Teil der Arbeit:
    # how to collect twitter corpus (conversations)
    # literatur von computerlinguisten sichten, die Twitter corpora erstellt haben. (tatjana Scheffler, Alan Ritter, andere?)
    # die literatur twitter corpora mit eigenem corpus vergleichen: quantitativ und qualitativ
        # wie viele tweets, wie viele sind davon mit emoji, wie viele davon wiederrum sind conversations

# 2. Teil der Arbeit:
    # Machine Learning